**Descsribe our project**

# About MIMIC dataset

**The overview of MIMIC data set: inlcuidng background, goal of MIMIC, content of the database**

MIMIC is a very large dataset, hence we only selected some part of it. We included the tables from four modules: core, ed, hosp, and icu:
* core: **Descsribe**
* ed:**Descsribe**
* hosp:**Descsribe**
* icu:**Descsribe**

We will take a glance at these modules first.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [12]:
# Load the data 
admissions = pd.read_csv('../data/core/admissions.csv')
patients = pd.read_csv('../data/core/patients.csv')
diagnose = pd.read_csv('../data/hosp/d_icd_diagnoses.csv')
subj_diagnose = pd.read_csv('../data/hosp/diagnoses_icd.csv')
vital_raw = pd.read_csv('../data/ed/vitalsign.csv')
icu_stays = pd.read_csv('../data/icu/icustays.csv')
item_names = pd.read_csv('../data/icu/d_items.csv').set_index('itemid')
chart_event = pd.read_csv('../data/icu/chart_event_filtered.csv')

## core module 

The most important table is the `patients` table where the information that is consistent for the lifetime of a patient is stored. This will be used to identify each patient. `dod` column contains the information about motality inside the hospital which will be used as target in this study.



In [4]:
patients.head(3)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000048,F,23,2126,2008 - 2010,NaN
1,10002723,F,0,2128,2017 - 2019,NaN
2,10003939,M,0,2184,2008 - 2010,NaN


**describe**

In [17]:
admissions.head(3)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0


## `hosp` module

Since we foucsed on ICU patients in this analysis, most of the tables in this module is not of our interest. 

The only information we need is the billed diagnosis of patients in order to select our cohort.

`diagnoses_icd.csv` contains the icd code for each subject and the corresponding icd version. With the code and version we can acquire the name of the diagnoses from the `d_icd_diagnoses.csv` file. 

In [8]:
subj_diagnose.head(3)

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,15734973,20475282,3,2825,9
1,15734973,20475282,2,V0251,9
2,15734973,20475282,5,V270,9


In [10]:
diagnose.head(3)

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"


## `ed` module

We only use `vitalsign` table in this module since vitalsigns like heart rate, respotary rate are directly related to death. And the first set of vital signs after patients admitted to emergency room is most important since they haven't received any medication. 

In [6]:
vital_raw.head(3)

,subject_id,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain
0,16113983,37539106,2116-06-10 00:32:00,98.2,82.0,15.0,NaN,106.0,72.0,NaN,NaN
1,15128994,30058281,2167-08-29 02:25:00,98.3,79.0,20.0,97.0,126.0,73.0,NaN,0
2,15128994,30058281,2167-08-29 04:51:00,97.6,79.0,20.0,98.0,126.0,73.0,NaN,0


## `icu` module

`icu` module contains the main data we are going to analyze. 

`icustays.csv` tracks information for ICU stays including adminission and discharge times. This table will be used to identify the patients who were admitted to the icu since not all patients admitted to ICU. 

In [11]:
icu_stays.head(3)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,17867402,24528534,31793211,Trauma SICU (TSICU),Trauma SICU (TSICU),2154-03-03 04:11:00,2154-03-04 18:16:56,1.587454
1,14435996,28960964,31983544,Trauma SICU (TSICU),Trauma SICU (TSICU),2150-06-19 17:57:00,2150-06-22 18:33:54,3.025625
2,17609946,27385897,33183475,Trauma SICU (TSICU),Trauma SICU (TSICU),2138-02-05 18:54:00,2138-02-15 12:42:05,9.741725


`chart_event_filtered.csv` contains the majority of the data. During their ICU stay, the primary repository of a patient’s information is their electronic chart. The electronic chart displays patients’ routine vital signs and any additional information relevant to their care: ventilator settings, laboratory values, code status, mental status, and so on. Each event is represented by an `item_id` in this table and the real name of the event can be found in the `d_items.csv` file

`chart_event` table is fairly large, so we uploaded a filtered version. The code used for filtering were shown in appendix. 

In [13]:
chart_event.head(3)

,Unnamed: 0,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,24,10007677,27180483,34992648,2120-11-21 12:00:00,2120-11-21 13:48:00,223900,Oriented,5.0,NaN,0
1,78,10007677,27180483,34992648,2120-11-17 20:00:00,2120-11-17 20:52:00,223900,Oriented,5.0,NaN,0
2,117,10007677,27180483,34992648,2120-11-21 18:48:00,2120-11-21 19:40:00,220645,130,130.0,mEq/L,1


In [14]:
item_names.head(3)

,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
itemid,,,,,,,,
220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN


# Select the Cohort 
Here we select the patients dignosed with ICH from the whole hospital addmission. The corresponding ICD codes are 'I61*' (ICD-10) and '431*' (ICD-9). 


Then we filter the patients who were addmitteed to the intensive care unit. 

In [ ]:
icu_ids = icu_stays['subject_id'].unique()

In [ ]:
def get_icu_cohort(icd10, icd9):
    """
    Input parameters are the icd code for a certain disease. For higher reliability, they should be regular expression. 
    """
    # filter all diagnosed subjects
    df1=patients.set_index('subject_id') 
    df_icd10=subj_diagnose[subj_diagnose['icd_code'].str.contains(str(icd10))]
    df_icd9 = subj_diagnose[subj_diagnose['icd_code'].str.contains(str(icd9))]
    df_icdall=pd.concat([df_icd9,df_icd10]).drop_duplicates('subject_id', keep='first')
    # filter the patients admitted to icu
    df_icu = df_icdall[df_icdall['subject_id'].isin(icu_ids)].set_index('subject_id')
    data = df_icu.join(df1,how='left')
    # add the target, i.e., mortality flag to the table
    data['dod']=data['dod'].replace(np.nan, 0, regex=True)
    data['dod']=data['dod'].replace('-', 1, regex=True)

    return data

def mortality_rate(df):
    """Calculate the mortality rate for selected cohort. This p
    return np.count_nonzero(df['dod']==1)/len(df['dod'])

In [ ]:
cohort = get_icu_cohort('I61',r'^431')
print('cohort size', cohort.shape[0])
print('mortality rate', mortality_rate(cohort))
cohort

# Select Features

In [ ]:
def get_data(cohort_index, data_table, index_colname):
    return data_table[data_table[index_colname].isin(cohort_index)]

In [ ]:
def fill_na_mean(df, rd=2, inplace=True):
    for col in list(df.columns[df.isnull().sum() > 0]):
        mean_val = df[col].mean()
        if inplace:
            try:
                df[col].fillna(round(mean_val,rd),inplace=inplace)
            except TypeError:
                continue
        else:
            try:
                df = df[col].fillna(round(mean_val,rd))
            except TypeError:
                continue
    if not inplace:
        return df

In [ ]:
cohort_ind = cohort.index

## Load Vital Signs




In [ ]:
vital_raw.head()

In [ ]:
# Drop rhythm, pain which have a lot of missing data
vital_id = get_data(cohort_ind, vital_raw, 'subject_id' ).iloc[:,:-2]
vital_id.head()

In [ ]:
fill_na_mean(vital_id)
vital_id

In [ ]:
vital_id =vital_id.drop_duplicates('subject_id', keep='last').set_index('subject_id')

In [ ]:
vital_id.head()

In [ ]:
final = cohort.join(vital_id)
final.shape

## Load chartted Event

In [ ]:
# Selected Features from table event

# PH (Venous)
# Chloride (serum)
# Creatinine (serum)
# Magnesium
# Sodium (serum)
# PH (dipstick)
# Sodium Bicarbonate 8.4%
# Norepinephrine
# Magnesium Sulfate
# PH (Arterial)
# GCS - Verbal Response
# Heparin Sodium
# Chloride (whole blood)
# CreatinineApacheIIScore
# CreatinineApacheIIValue
# HematocritApacheIIScore
# WbcApacheIIScore
# WBCApacheIIValue
# Hematocrit_ApacheIV
# HR_ApacheIV
# WBC_ApacheIV
# WBCScore_ApacheIV
# Magnesium Sulfate (Bolus)
# Chloride (serum) (soft)
# Sodium (serum) (soft)
# Creatinine (whole blood)
# selected_items = [220274,220602,220615,220635,220645,220734,220995,221906,222011,223830,223900,
# 225152,226536,226751,226752,226761,226779,226780,227017,227018,227062,227063,227523,228385,228389,229761,]

In [ ]:
# df[df['itemid'].isin(selected_items)].to_csv('chart_event_filtered.csv')

In [ ]:
# len(df['itemid'].unique())

In [ ]:
charteve_raw.head()

In [ ]:
item_names.head()

In [ ]:
item_dict = dict()
for item_id in charteve_raw['itemid'].unique():
    item_dict[item_id] = item_names.loc[item_id, 'abbreviation']

In [ ]:
# This command takes a few minutes to run.
for i in range(charteve_raw.shape[0]):
    entry = charteve_raw.iloc[i,:]
    sub_id, itemid = entry[['subject_id', 'itemid']]
    item_names = item_dict[itemid]
    final.loc[sub_id, item_names] = entry['valuenum']
final.head() 

In [ ]:
test = final.iloc[:,:-5].dropna(how='any',axis=0)
test[['dod']] = test[['dod']].astype('int')
test.describe()

In [ ]:
g = sns.catplot(x='dod',y='Sodium (serum)',data=test, kind="box",
                height=4, aspect=1);

# Description

# Modelling

In [ ]:
from DS_MIMIC_knn import shuffle_split_data

In [ ]:
test[test.dod==0].shape[0]/test.shape[0]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

X = test[['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp',
       'Verbal Response', 'Sodium (serum)', 'Chloride (serum)', 'Magnesium',
       'Creatinine (serum)']]
y = test['dod']

X_train_orig, X_test_orig, y_train, y_test = shuffle_split_data(X, y,0.2)
std= StandardScaler()
X_train=std.fit_transform(X_train_orig)
X_test=std.fit_transform(X_test_orig)

In [ ]:
X_train.shape

## Logistic Regression

In [ ]:
# check classification scores of logistic regression
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
print('Train/Test split results:')
print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

In [ ]:
idx = np.min(np.where(tpr > 0.95)) # index of the first threshold for which the sensibility > 0.95

plt.figure()
plt.plot(fpr, tpr, color='coral', label='ROC curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot([0,fpr[idx]], [tpr[idx],tpr[idx]], 'k--', color='blue')
plt.plot([fpr[idx],fpr[idx]], [0,tpr[idx]], 'k--', color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

print("Using a threshold of %.3f " % thr[idx] + "guarantees a sensitivity of %.3f " % tpr[idx] +  
      "and a specificity of %.3f" % (1-fpr[idx]) + 
      ", i.e. a false positive rate of %.2f%%." % (np.array(fpr[idx])*100))

## Random Forest

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Fit the model
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train_orig, y_train)

# Use score method to calculate the accuracy over the whole test set
acc=rf.score(X_test_orig, y_test)
print(acc)
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X, y)


ft_imp = pd.Series(rf.feature_importances_).sort_values(ascending=False)
ft_imp.head(10)
ft_imp

In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(rf)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(X_test_orig)

# Make plot. Index of [1] is explained in text below.
shap.summary_plot(shap_values[1], X_test_orig)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(40)
test_scores = []
train_scores = []

for i in range(1,15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))


In [ ]:
max_train_score = max(train_scores)
train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
print('Max train score {} % and k = {}'.format(max_train_score*100,list(map(lambda x: x+1, train_scores_ind))))

In [ ]:
## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

In [ ]:
plt.figure(figsize=(12,5))
p = sns.lineplot(x=range(1,15),y=train_scores,marker='*',label='Train Score')
p = sns.lineplot(x=range(1,15),y=test_scores,marker='o',label='Test Score')

## My KNN


In [ ]:
def My_knncomper_difftest( X_train, X_test, y_train, y_test,kmax):
    rightnumber = 0           
    print('enter phase 2')
    Train_acc=[]
    Test_acc=[]
    print(" maximum K value is "+str(kmax))           
    for k in range(1,kmax):
        test_predict_score=0
        rightnumber=0
        predictlist =MYknn(X_test, X_train, y_train, k)
        for m in range(len(X_test)):
            if predictlist[m] == y_test[m]:
                rightnumber = rightnumber + 1
        test_predict_score = (rightnumber / len(y_test))
        print(test_predict_score*100)
        Test_acc.append(  test_predict_score * 100)
        rightnumber=0
        Train_predict_score=0
        predictlist = MYknn(X_train, X_train, y_train, k)
        for m in range(len(X_train)):
            if predictlist[m] == y_train[m]:
                rightnumber = rightnumber + 1
        Train_predict_score = (rightnumber / len(X_train))
        Train_acc.append( Train_predict_score * 100)      

    plt.plot(Test_acc)
    plt.title('Test_acc')
    plt.show()
    plt.plot(Train_acc)
    plt.title('Train_acc')
    plt.show()

   


In [ ]:
def MYknn(test_object, training_object, training_object_target, K):
    predictlist = []
    for newpoint in test_object:
        dataSetSize = training_object.shape[0]
        diffMat= np.tile(newpoint, (dataSetSize, 1)) - training_object
        sqDistances = (diffMat**2).sum(axis=1)
        distances = sqDistances ** 0.5
        sortedDistIndicies = distances.argsort()
        classCount = {}
        for i in range(K):
            voteIlabel = training_object_target[sortedDistIndicies[i]]
            classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1  #Get the value of key from the map and return 0 without key
            sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
        predictlist.append(sortedClassCount[0][0])
    return predictlist

In [ ]:
My_knncomper_difftest(X_train, X_test, y_train, y_test, 15)

In [ ]:
## from sklearn.model_selection import train_test_split
# from DS_MIMIC_knn import *
import numpy as np
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler




---------------------
# Find dod ID


In [ ]:
raise TypeError

In [ ]:
# fillnan_test.set_index('subject_id',inplae=True)

In [ ]:
# dod_id=cohort[cohort['dod']==1]
# test = dod_id.join(fillnan_test,how='left')
# np.count_nonzero(test['stay_id']>1)

In [ ]:
dod_id

In [ ]:
test['stay_id']

# Load output event

In [ ]:
outevents = pd.read_csv('../data/icu/outputevents.csv')
itmes = pd.read_csv('../data/icu/d_items.csv')

In [ ]:
test_main = cohort.copy()

In [ ]:
out = outevents[outevents['subject_id'].isin(test_main.index)]

In [ ]:
for i in range(out.shape[0]):
    entry = out.iloc[i,:]
    sub_id, itemid = entry[['subject_id', 'itemid']]
    test_main.loc[sub_id, itemid] = entry['value']
test_main.head() 

# Appendix

We select chart event data from Google Bigquery using SQL. This command took about 20 mins to run, and returns milions of rows of data.
So won't run it in the notebook, we filtered the data and saved it locally. 

In [ ]:
for i in range(5):
    sql = f"""SELECT * 
    FROM `physionet-data.mimic_icu.chartevents`
    WHERE subject_id in {tuple(cohort.index[497*i:497*(i+1)].values.tolist())}
    ORDER BY subject_id"""

    df = pd.read_gbq(sql, project_id='focus-dragon-313813', dialect='standard', use_bqstorage_api=True)
    df.to_csv(f'df{i}.csv')